In [28]:
!pip install transformers datasets evaluate
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 663.0 kB/s eta 0:00:00a 0:00:01


In [4]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /home/tomer/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
books = books["train"].train_test_split(test_size=0.2)

In [8]:
books["train"][0]

{'id': '121613',
 'translation': {'en': "The wind blew between the planks of chestnut and oak, and they rolled themselves up as in some wood-cutter's abandoned hut.",
  'fr': 'Le vent sifflait entre les perches de châtaignier et de chene, ils se pelotonnaient, comme dans une hutte de bucheron abandonnée.'}}

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [17]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [18]:
tokenized_books = books.map(preprocess_function, batched=True)

  0%|          | 0/102 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/tomer/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf/cache-191295a3d1332415.arrow


In [20]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [29]:
import evaluate

sacrebleu = evaluate.load("sacrebleu")

In [24]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation, id. If translation, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 101668
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25418
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss


Saving model checkpoint to my_awesome_opus_books_model/checkpoint-500
Configuration saved in my_awesome_opus_books_model/checkpoint-500/config.json
Configuration saved in my_awesome_opus_books_model/checkpoint-500/generation_config.json
Model weights saved in my_awesome_opus_books_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in my_awesome_opus_books_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in my_awesome_opus_books_model/checkpoint-500/special_tokens_map.json
Saving model checkpoint to my_awesome_opus_books_model/checkpoint-1000
Configuration saved in my_awesome_opus_books_model/checkpoint-1000/config.json
Configuration saved in my_awesome_opus_books_model/checkpoint-1000/generation_config.json
Model weights saved in my_awesome_opus_books_model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in my_awesome_opus_books_model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in my_awesome_opus_books_model/checkpoi

Deleting older checkpoint [my_awesome_opus_books_model/checkpoint-5500] due to args.save_total_limit
Saving model checkpoint to my_awesome_opus_books_model/checkpoint-7500
Configuration saved in my_awesome_opus_books_model/checkpoint-7500/config.json
Configuration saved in my_awesome_opus_books_model/checkpoint-7500/generation_config.json
Model weights saved in my_awesome_opus_books_model/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in my_awesome_opus_books_model/checkpoint-7500/tokenizer_config.json
Special tokens file saved in my_awesome_opus_books_model/checkpoint-7500/special_tokens_map.json
Deleting older checkpoint [my_awesome_opus_books_model/checkpoint-6000] due to args.save_total_limit
Saving model checkpoint to my_awesome_opus_books_model/checkpoint-8000
Configuration saved in my_awesome_opus_books_model/checkpoint-8000/config.json
Configuration saved in my_awesome_opus_books_model/checkpoint-8000/generation_config.json
Model weights saved in my_awesome_opus_